In [1]:
import pandas as pd
import numpy as np

In [2]:
from google.colab import files
data_to_load = files.upload()

Saving test_nltk.csv to test_nltk (1).csv
Saving entrenamiento_nltk.csv to entrenamiento_nltk (1).csv


In [3]:
data_to_load = files.upload()

In [4]:
train = pd.read_csv('entrenamiento_nltk.csv', converters={'text': eval})
test = pd.read_csv('test_nltk.csv', converters={'text': eval})

In [5]:
train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,"[our, deed, reason, earthquak, may, allah, for...",1
1,4,NaN,NaN,"[forest, fire, near, la, rong, sask, canada]",1
2,5,NaN,NaN,"[all, resid, ask, shelter, place, notifi, offi...",1
3,6,NaN,NaN,"[13, 000, peopl, receiv, wildfir, evacu, order...",1
4,7,NaN,NaN,"[just, got, sent, photo, rubi, alaska, smoke, ...",1


In [11]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [8]:
X = train['text']
y = train['target']
le = LabelEncoder()
y = le.fit_transform(y)
y = y.reshape(-1,1)

In [9]:
y

array([[1],
       [1],
       [1],
       ...,
       [1],
       [1],
       [1]])

In [12]:
X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.2, random_state=123)

In [13]:
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping

In [14]:
max_words = 1500
max_len = 100
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

In [15]:
sequences_matrix

array([[  0,   0,   0, ...,  29,   2,   1],
       [  0,   0,   0, ..., 707, 676, 242],
       [  0,   0,   0, ...,   1,   2,   1],
       ...,
       [  0,   0,   0, ..., 419, 120, 828],
       [  0,   0,   0, ..., 148, 263, 201],
       [  0,   0,   0, ..., 837,   2,   1]], dtype=int32)

In [16]:
# Red neuronal RNN
inputs = Input(name='inputs',shape=[max_len])
layer = Embedding(max_words,100,input_length=max_len)(inputs)
layer = LSTM(128)(layer)
layer = Dense(128)(layer)
layer = Activation('relu')(layer)
layer = Dropout(0.5)(layer)
layer = Dense(1)(layer)
layer = Activation('sigmoid')(layer)
model = Model(inputs=inputs,outputs=layer)

In [17]:
layer

<tf.Tensor 'activation_1/Sigmoid:0' shape=(None, 1) dtype=float32>

In [18]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [19]:
model.fit(sequences_matrix,y_train,batch_size=32,epochs=5,
          validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.001)])

Epoch 1/5
153/153 [==============================] - 20s 132ms/step - loss: 0.5616 - accuracy: 0.7110 - val_loss: 0.4512 - val_accuracy: 0.8120
Epoch 2/5
153/153 [==============================] - 20s 131ms/step - loss: 0.3923 - accuracy: 0.8259 - val_loss: 0.4583 - val_accuracy: 0.8030


In [20]:
test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)

In [21]:
# Accuracy
model.evaluate(test_sequences_matrix,y_test)

48/48 [==============================] - 2s 33ms/step - loss: 0.4579 - accuracy: 0.7925


[0.45791953802108765, 0.7925148010253906]

In [24]:
tok.fit_on_texts(X)
sequences = tok.texts_to_sequences(X)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

In [25]:
model.fit(sequences_matrix,y,batch_size=32,epochs=5,
          validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.001)])

Epoch 1/5
191/191 [==============================] - 25s 131ms/step - loss: 0.5667 - accuracy: 0.7071 - val_loss: 0.4655 - val_accuracy: 0.7833
Epoch 2/5
191/191 [==============================] - 25s 131ms/step - loss: 0.4157 - accuracy: 0.8174 - val_loss: 0.4441 - val_accuracy: 0.7997
Epoch 3/5
191/191 [==============================] - 25s 130ms/step - loss: 0.3579 - accuracy: 0.8537 - val_loss: 0.4571 - val_accuracy: 0.7912


In [26]:
test.head()

,id,keyword,location,text
0,0,NaN,NaN,"[just, happen, terribl, car, crash]"
1,2,NaN,NaN,"[heard, earthquak, differ, citi, stay, safe, e..."
2,3,NaN,NaN,"[forest, fire, spot, pond, gees, flee, across,..."
3,9,NaN,NaN,"[apocalyps, light, spokan, wildfir]"
4,11,NaN,NaN,"[typhoon, soudelor, kill, 28, china, taiwan]"


In [27]:
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(test['text'])
sequences = tok.texts_to_sequences(test['text'])
sequences_matrix_test = sequence.pad_sequences(sequences,maxlen=max_len)

In [39]:
preds = model.predict(sequences_matrix_test)

In [40]:
preds

array([[0.184791  ],
       [0.9946943 ],
       [0.6331171 ],
       ...,
       [0.85962415],
       [0.15033492],
       [0.23805627]], dtype=float32)

In [41]:
preds.shape

(3263, 1)

In [42]:
preds = preds.reshape(-1)

In [43]:
preds

array([0.184791  , 0.9946943 , 0.6331171 , ..., 0.85962415, 0.15033492,
       0.23805627], dtype=float32)

In [44]:
target = pd.DataFrame({'id':test['id'], 'target':preds})

In [45]:
target.head()

,id,target
0,0,0.184791
1,2,0.994694
2,3,0.633117
3,9,0.231883
4,11,0.320158


In [46]:
target['target'] = target['target'].agg(lambda x: 0 if (x<0.5) else 1)

In [47]:
target['target'].value_counts()

0    1964
1    1299
Name: target, dtype: int64

In [48]:
target.head()

,id,target
0,0,0
1,2,1
2,3,1
3,9,0
4,11,0


In [49]:
target.to_csv('submit_embedding_lstm_3.csv', index=False)